# Jacobian Derivation for BAL

## Set Up Session

In [1]:
# set up session:
from sympy import *
# init_printing(use_unicode=True)

## Define Variables

In [2]:
# projection in camera frame:
x, y, z = symbols('x y z')
# camera intrinsics & distortion:
f, k1, k2 = symbols('f k1 k2')
# measurement:
u0, v0 = symbols('u0 v0')

## Camera Projection with Distortion

In [3]:
# projection on normalized plane:
x_prime = -(x / z)
y_prime = -(y / z)

# distortion factor:
r_square = (x_prime**2 + y_prime**2)
distortion_factor = 1 + r_square*(k1 + k2*r_square)

# projection on pixel plane:
u = f*distortion_factor*x_prime
v = f*distortion_factor*y_prime

# error:
eu = u0 - u
ev = v0 - v

## Jacobian

In [4]:
# Jacobian of pixel plane projection with respect to projection on normalized plane:
J_p_prime = Matrix(
    [
        [diff(eu, x), diff(eu, y), diff(eu, z)],
        [diff(ev, x), diff(ev, y), diff(ev, z)]
    ]
)
# Jacobian of normalized plane projection with respect to se3:
J_lie = -Matrix(
    [
        [+0, -z, +y],
        [+z, +0, -x],
        [-y, +x, +0]
    ]
)
# Jacobian of pixel plane projection with respect to camera intrinsics and radial distortions
J_camera = Matrix(
    [
        [diff(eu, f), diff(eu, k1), diff(eu, k2)],
        [diff(ev, f), diff(ev, k1), diff(ev, k2)]
    ]
)
# dim 0~2 of CameraBAL:
J_rotation = simplify(J_p_prime * J_lie)
# dim 3~5 of CameraBAL:
J_translation = simplify(J_p_prime)
# dim 6~8 of CameraBAL:
J_projection = simplify(J_camera)

# final Jacobians:
J_xi = Matrix(
    [
        (J_rotation.row(0).tolist()[0] + J_translation.row(0).tolist()[0] + J_projection.row(0).tolist()[0]),
        (J_rotation.row(1).tolist()[0] + J_translation.row(1).tolist()[0] + J_projection.row(1).tolist()[0]),        
    ]
)
J_xj = simplify(J_p_prime)

## Convert to Ready-to-Use Format for G2O

In [5]:
def convert_to_g2o_code(J, vertex_name):
    """ convert matrix in sympy to g2o c++ code
    """
    result = []

    for i in range(2):
        desc = str(J.row(i).tolist()[0])
        # orders:
        for k in range(2, 7):
            # variables:
            for v in 'xyz':
                src = "{v}**{k}".format(v=v, k=k)
                dst = "{v}_{k}".format(v=v, k=k)

                desc = desc.replace(src, dst)
        # radial:
        desc = desc.replace('(x_2 + y_2)', 'r_2')
        desc = desc.replace('r_2**2', 'r_4')
        
        result.append(
            "\n".join(
                [
                    "_jacobianOplusX{vertex_name}({i},{j}) = {v};".format(
                        vertex_name=vertex_name,
                        i=i, j=j, 
                        v=v
                    ) 
                    for j,v in enumerate(desc[1:-1].split(','))
                ]
            )
        )

    return "\n\n".join(result)

In [6]:
# vertex_camera:
print(convert_to_g2o_code(J_xi, 'i'))

_jacobianOplusXi(0,0) = -f*x*y*(z_4 + 2*z_2*(k1*z_2 + 2*k2*r_2) + r_2*(k1*z_2 + k2*r_2) + 2*r_2*(k1*z_2 + 2*k2*r_2))/z_6;
_jacobianOplusXi(0,1) =  f*(x_2*(z_4 + r_2*(k1*z_2 + k2*r_2) + 2*r_2*(k1*z_2 + 2*k2*r_2)) + z_2*(2*x_2*(k1*z_2 + 2*k2*r_2) + z_4 + r_2*(k1*z_2 + k2*r_2)))/z_6;
_jacobianOplusXi(0,2) =  -f*y*(z_4 + r_2*(k1*z_2 + k2*r_2))/z_5;
_jacobianOplusXi(0,3) =  f*(2*x_2*(k1*z_2 + 2*k2*r_2) + z_4 + r_2*(k1*z_2 + k2*r_2))/z_5;
_jacobianOplusXi(0,4) =  2*f*x*y*(k1*z_2 + 2*k2*r_2)/z_5;
_jacobianOplusXi(0,5) =  -f*x*(z_4 + r_2*(k1*z_2 + k2*r_2) + 2*r_2*(k1*z_2 + 2*k2*r_2))/z_6;
_jacobianOplusXi(0,6) =  x*(z_4 + r_2*(k1*z_2 + k2*r_2))/z_5;
_jacobianOplusXi(0,7) =  f*x*r_2/z_3;
_jacobianOplusXi(0,8) =  f*x*r_4/z_5;

_jacobianOplusXi(1,0) = -f*(y_2*(z_4 + r_2*(k1*z_2 + k2*r_2) + 2*r_2*(k1*z_2 + 2*k2*r_2)) + z_2*(2*y_2*(k1*z_2 + 2*k2*r_2) + z_4 + r_2*(k1*z_2 + k2*r_2)))/z_6;
_jacobianOplusXi(1,1) =  f*x*y*(z_4 + 2*z_2*(k1*z_2 + 2*k2*r_2) + r_2*(k1*z_2 + k2*r_2) + 2*r_2*(k1*z_2 + 2*k2*r_

In [7]:
# vertex_point, partial:
print(convert_to_g2o_code(J_xj, 'j'))

_jacobianOplusXj(0,0) = f*(2*x_2*(k1*z_2 + 2*k2*r_2) + z_4 + r_2*(k1*z_2 + k2*r_2))/z_5;
_jacobianOplusXj(0,1) =  2*f*x*y*(k1*z_2 + 2*k2*r_2)/z_5;
_jacobianOplusXj(0,2) =  -f*x*(z_4 + r_2*(k1*z_2 + k2*r_2) + 2*r_2*(k1*z_2 + 2*k2*r_2))/z_6;

_jacobianOplusXj(1,0) = 2*f*x*y*(k1*z_2 + 2*k2*r_2)/z_5;
_jacobianOplusXj(1,1) =  f*(2*y_2*(k1*z_2 + 2*k2*r_2) + z_4 + r_2*(k1*z_2 + k2*r_2))/z_5;
_jacobianOplusXj(1,2) =  -f*y*(z_4 + r_2*(k1*z_2 + k2*r_2) + 2*r_2*(k1*z_2 + 2*k2*r_2))/z_6;
